In [ ]:
# Import pandas 
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the customer_data
customer_data = pd.read_csv('datasets/Customer_dataset.csv',
                            parse_dates=True,
                            infer_datetime_format=True)

# Load the app_purchases
app_purchases = pd.read_csv('datasets/inapp_purchases_dataset.csv',
                            parse_dates=True,
                            infer_datetime_format=True)
# Load the daily_revenue
daily_revenue = pd.read_csv('datasets/Daily_revenue_dataset.csv')

# Load  the user_purchases 
paywal_data = pd.read_csv('datasets/User_demographics_paywal_dataset.csv')

# Print the columns of customer data
print(customer_data.columns)

# Print the columns of app_purchases
print(app_purchases.columns)

# Print the columns of customer data
print(daily_revenue.columns)

# Print the columns of app_purchases
print(paywal.columns)

### Experimental units: Revenue per user day
We are going to check what happens when we add a consumable paywall to our app. A paywall is a feature of a website or other technology that requires payment from users in order to access additional content or services.

Here, you'll practice calculating experimental units and baseline values related to our consumable paywall. Both measure revenue only among users who viewed a paywall. Your job is to calculate revenue per user-day, with user-day as the experimental unit.

Note: paywal_data's columns are different from course dataset's columns. (I took datasets from Datacamp course. But day misload paywal_data dataset.) Beacuse of that I don't run my codes related paywal_data dataset.  

In [ ]:
# Join our paywall wiews to the user demographics:
purchase_data = customer_data.merge(paywal_data, how ='left', on=['uid', 'reg_date'])

In [ ]:
# Changing reg_date column type from string to datetime:
purchase_data["reg_date"] = pd.to_datetime(purchase_data.reg_date, format = '%Y-%m-%d', errors='coerce')  

In [ ]:
# Extract the 'day'; value from the timestamp
purchase_data.reg_date = purchase_data.reg_date.dt.floor('d')

# Replace the NaN price values with 0 
purchase_data.price = np.where(np.isnan(purchase_data.price), 0, purchase_data.price)

# Aggregate the data by 'uid' & 'date'
purchase_data_agg = purchase_data.groupby(by=['uid', 'reg_date'], as_index=False)
revenue_user_day = purchase_data_agg.sum()

# Calculate the final average
revenue_user_day = revenue_user_day.price.mean()
print(revenue_user_day)

### Conversion rate sensitivities
To mix things up, we will spend the next few exercises working with the conversion rate metric we explored in Chapter One. Specifically you will work to examine what that value becomes under different percentage lifts and look at how many more conversions per day this change would result in. First you will find the average number of paywall views and purchases that were made per day in our observed sample. Good luck!

In [ ]:
# Merge and group the datasets
purchase_data = customer_data.merge(paywal_data,  how='inner', on=['uid'])
purchase_data.date = purchase_data.date.dt.floor('d')

# Group and aggregate our combined dataset 
daily_purchase_data = purchase_data.groupby(by=['date'], as_index=False)
daily_purchase_data = daily_purchase_data.agg({'purchase': ['sum', 'count']})

# Find the mean of each field and then multiply by 1000 to scale the result
daily_purchases = daily_purchase_data.purchase['sum'].mean()
daily_paywall_views = daily_purchase_data.purchase['count'].mean()
daily_purchases = daily_purchases * 0.1
daily_paywall_views = daily_paywall_views * 1000

print(daily_purchases)
print(daily_paywall_views)

### Sensitivity
Continuing with the conversion rate metric, you will now utilize the results from the previous exercise to evaluate a few potential sensitivities that we could make use of in planning our experiment. The baseline conversion_rate has been loaded for you, calculated in the same way we saw in Chapter One. Additionally the daily_paywall_views and daily_purchases values you calculated previously have been loaded.

In [ ]:
small_sensitivity = 0.1 

# Find the conversion rate when increased by the percentage of the sensitivity above
small_conversion_rate = conversion_rate * (1 + small_sensitivity) 

# Apply the new conversion rate to find how many more users per day that translates to
small_purchasers = daily_paywall_views * small_conversion_rate

# Subtract the initial daily_purcahsers number from this new value to see the lift
purchaser_lift = small_purchasers - daily_purchases

print(small_conversion_rate)
print(small_purchasers)
print(purchaser_lift)

In [ ]:
medium_sensitivity = 0.2

# Find the conversion rate when increased by the percentage of the sensitivity above
medium_conversion_rate = conversion_rate * (1 + medium_sensitivity) 

# Apply the new conversion rate to find how many more users per day that translates to
medium_purchasers = daily_paywall_views * medium_conversion_rate

# Subtract the initial daily_purcahsers number from this new value to see the lift
purchaser_lift = medium_purchasers - daily_purchases

print(medium_conversion_rate)
print(medium_purchasers)
print(purchaser_lift)

In [ ]:
large_sensitivity = 0.5

# Find the conversion rate lift with the sensitivity above
large_conversion_rate = conversion_rate * (1 + large_sensitivity)

# Find how many more users per day that translates to
large_purchasers = daily_paywall_views * large_conversion_rate
purchaser_lift = large_purchasers - daily_purchases

print(large_conversion_rate)
print(large_purchasers)
print(purchaser_lift)

### Standard deviation:
We can find the standard deviation of our data using the pandas `std()` method by passing in a vector of our statistics.

In [ ]:
# Calculate the standard deviation of revenue per user
revenue_variation = total_revenue.price.std()
print(revenue_variation)

### Standard error
Previously we observed how to calculate the standard deviation using the .std() method. In this exercise, you will explore how to calculate standard deviation for a conversion rate, which requires a slightly different procedure. You will calculate this step by step in this exercise.

Loaded for you is our inner merged dataset purchase_data as well as the computed conversion_rate value.

In [ ]:
# Find the n & v quantities
n = purchase_data.purchase.count()

# Calculate the quantity "v"
v = conversion_rate * (1 - conversion_rate) 

# Calculate the variance and standard error of the estimate
var = v / n 
se = var**0.5

print(var)
print(se)

### Exploring the power calculation
As discussed, power is the probability of rejecting the null hypothesis when the alternative hypothesis is true. Here you will explore some properties of the power function and see how it relates to sample size among other parameters. The get_power() function has been included and takes the following arguments in the listed order n for sample size, p1 as the baseline value, p2 as the value with lift included, and cl as the confidence level.

In [ ]:
#Calculate the test power (some details omitted)
def get_power(n, p1, p2, cl):
    alpha = 1 - cl
    qu = stats.norm.ppf(1 - alpha / 2)
    diff = abs(p2 - p1)
    bp = (p1 + p2) / 2
    
    power = power_part_one + power_part_two
    
    return(power)

In [ ]:
# Look at the impact of sample size increase on power
n_param_one = get_power(n=1000, p1=p1, p2=p2, cl=cl)
n_param_two = get_power(n=2000, p1=p1, p2=p2, cl=cl)

# Look at the impact of confidence level increase on power
alpha_param_one = get_power(n=n1, p1=p1, p2=p2, cl=0.8)
alpha_param_two = get_power(n=n1, p1=p1, p2=p2, cl=0.95)
    
# Compare the ratios
print(n_param_two / n_param_one)
print(alpha_param_one / alpha_param_two)

### Calculating the sample size
You're now going to utilize the sample size function to determine how many users you need for the test and control groups under various circumstances.

Included is the get_sample_size() function you viewed previously, which takes four primary arguments, power, p1, p2 and cl as described before:

`def get_sample_size(power, p1, p2, cl, max_n=1000000):
    
    n = 1 
    
    while n <= max_n:
    
        tmp_power = get_power(n, p1, p2, cl)

        if tmp_power >= power: 
       
           return n 
        
        else: 
        
            n = n + 100

    return "Increase Max N Value" `
You will continue working with the paywall conversion rate data for this exercise, which has been pre-loaded as purchase_data.

In [ ]:
def get_sample_size(power, p1, p2, cl, max_n=1000000):
    n = 1 
    while n <= max_n:
        tmp_power = get_power(n, p1, p2, cl)

        if tmp_power >= power: 
               return n 
        else: 
            n = n + 100

    return "Increase Max N Value"

In [ ]:
# Merge the demographics and purchase data to only include paywall views
purchase_data = customer_data.merge(paywal_data, how='inner', on=['uid'])
                            
# Find the conversion rate
conversion_rate = (sum(purchase_data.purchase) / purchase_data.purchase.count())
            
print(conversion_rate)

In [ ]:
# Merge the demographics and purchase data to only include paywall views
purchase_data = customer_data.merge(paywal_data, how='inner', on=['uid'])
                            
# Find the conversion rate
conversion_rate = (sum(purchase_data.purchase) / purchase_data.purchase.count())

# Desired Power: 0.8
# CL: 0.90
# Percent Lift: 0.1
p2 = conversion_rate * (1 + 0.1)
sample_size = get_sample_size(0.8, conversion_rate, p2, 0.90)
print(sample_size)

In [ ]:
# Find the conversion rate
conversion_rate = (sum(purchase_data.purchase) / purchase_data.purchase.count())

# Desired Power: 0.95
# CL 0.90
# Percent Lift: 0.1
p2 = conversion_rate * (1 + 0.1)
sample_size = get_sample_size(0.95, conversion_rate, p2, 0.90)
print(sample_size)